In [52]:
# HW4 is about machine learning algorthms and cleaning data
#--------
# notes to take on HW4:
# - pg.63: 3 ways to clean data and methods to do so
# - pg.63-64: how to replace missing all values to the median and
# back into the pandas dataframe
# - pg.64-65: scikit-learn design
# - pg.66: what/how is used to convert text to numbers
# - pg.66: why use fit_transform() for training, and transform() for testing
# - pg.67: what is one-hot encoding (whats a sparse matrix)
# - pg.68: how to create a custom transformers
# - pg.69: hyperparameter
# - pg.69: how to have attributes to have the same scale
# - pg.70: what is a pipeline good for?
# - pg.70: what exactly does the fit_transform() do?
# - what are tuples?
# - pg.72: about rmse and why mean_squared_error is useful
# - pg.72-73: what does underfitting look like and a way to fix it
# - pg.73: what could overfitting data look like?
# $ - pg.73: what's cross-validation
# - pg.74: whats the difference between a utility and a cost function(x-valid)\
# - steps to creating and testing a model
# - pg.75: what is joblib
# - pg.76: what is a gridsearch and why use GridSearchCV
# - pg.78: what is randomizedSearchCV and why might it be useful
# - pg.78: what are ensemble models
# - pg.78: what is a best_estimator
# - pg.78: what does feature_importance do
# - scientific notation
# - pg.81: how could the model be used
# - pg.81: what is model rot
# - 
# ----- general notes: -----
# - Since Scikit-Learn 0.20, the sklearn.preprocessing.Imputer class 
# was replaced by the sklearn.impute.SimpleImputer class.

In [156]:
import numpy as np
import urllib
import sklearn
import pandas as pd
import os
import tarfile
import joblib
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
from scipy import stats

In [15]:
# getting data
DATA_ROOT = "https://raw.githubusercontent.com/ageron/handson-ml2/master/"
HOUSING_URL = DATA_ROOT + "datasets/housing/housing.tgz"
HOUSING_PATH = os.path.join("datasets", "housing")

def fetch_data(housing_url = HOUSING_URL, housing_path = HOUSING_PATH):
    os.makedirs(housing_path, exist_ok = True)
    tgz_path = os.path.join(housing_path, 'housing.tgz')
    urllib.request.urlretrieve(housing_url, tgz_path)
    housing_tgz = tarfile.open(tgz_path)
    housing_tgz.extractall(path=housing_path)
    housing_tgz.close()

fetch_data()

def load_data(housing_path = HOUSING_PATH):
    csv_path = os.path.join(housing_path, 'housing.csv')
    return pd.read_csv(csv_path)

housing = load_data()
housing.head()
housing.describe()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value
count,20640.000000,20640.000000,20640.000000,20640.000000,20433.000000,20640.000000,20640.000000,20640.000000,20640.000000
mean,-119.569704,35.631861,28.639486,2635.763081,537.870553,1425.476744,499.539680,3.870671,206855.816909
std,2.003532,2.135952,12.585558,2181.615252,421.385070,1132.462122,382.329753,1.899822,115395.615874
min,-124.350000,32.540000,1.000000,2.000000,1.000000,3.000000,1.000000,0.499900,14999.000000
25%,-121.800000,33.930000,18.000000,1447.750000,296.000000,787.000000,280.000000,2.563400,119600.000000
50%,-118.490000,34.260000,29.000000,2127.000000,435.000000,1166.000000,409.000000,3.534800,179700.000000
75%,-118.010000,37.710000,37.000000,3148.000000,647.000000,1725.000000,605.000000,4.743250,264725.000000
max,-114.310000,41.950000,52.000000,39320.000000,6445.000000,35682.000000,6082.000000,15.000100,500001.000000


In [16]:
# creating training and testing indexes (sets) using housing[income_cat]
housing["income_cat"] = pd.cut(housing["median_income"],
                              bins = [0., 1.5, 3.0, 4.5, 6.0, np.inf],
                              labels=[1, 2, 3, 4, 5])

split = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
for train_index, test_index in split.split(housing, housing["income_cat"]):
    strat_train_set = housing.loc[train_index]
    strat_test_set = housing.loc[test_index]

In [17]:
# dropping housing data set and copying the data
# would be used to ensure data is unaffected when adjusting and learning data
# basic: getting rid of housing and making housing_labels
housing = strat_train_set.drop('median_house_value', axis=1)
housing_labels = strat_train_set['median_house_value'].copy()

In [20]:
housing_cat = housing['ocean_proximity']
housing_cat.head(10)

12655        INLAND
15502    NEAR OCEAN
2908         INLAND
14053    NEAR OCEAN
20496     <1H OCEAN
1481       NEAR BAY
18125     <1H OCEAN
5830      <1H OCEAN
17989     <1H OCEAN
4861      <1H OCEAN
Name: ocean_proximity, dtype: object

In [21]:
# notice the difference using [[]] instead of []
housing_cat = housing[['ocean_proximity']]
housing_cat.head(10)

,ocean_proximity
12655,INLAND
15502,NEAR OCEAN
2908,INLAND
14053,NEAR OCEAN
20496,<1H OCEAN
1481,NEAR BAY
18125,<1H OCEAN
5830,<1H OCEAN
17989,<1H OCEAN
4861,<1H OCEAN


In [27]:
# converting text to numbers with ordinal encoder
ordinal_encoder = OrdinalEncoder()
# The fit method is calculating the mean and variance of 
# each of the features present in our data. The transform method is transforming 
# all the features using the respective mean and variance.
housing_cat_encoded = ordinal_encoder.fit_transform(housing_cat)
housing_cat_encoded[:10]

array([[1.],
       [4.],
       [1.],
       [4.],
       [0.],
       [3.],
       [0.],
       [0.],
       [0.],
       [0.]])

In [28]:
# to get a list of all the attributes above we can use categories_
ordinal_encoder.categories_

[array(['<1H OCEAN', 'INLAND', 'ISLAND', 'NEAR BAY', 'NEAR OCEAN'],
       dtype=object)]

In [30]:
# applying one-hot encoder to the attributes above
#----
# this creates a SciPy sparse matrix 
cat_encoder = OneHotEncoder()
housing_cat_1hot = cat_encoder.fit_transform(housing_cat)
housing_cat_1hot

<16512x5 sparse matrix of type '<class 'numpy.float64'>'
	with 16512 stored elements in Compressed Sparse Row format>

In [32]:
# if we want to convert the above into a dense numpy 2d matrix
housing_cat_1hot.toarray()

array([[0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 1.],
       [0., 1., 0., 0., 0.],
       ...,
       [1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0.]])

In [39]:
housing.columns

Index(['longitude', 'latitude', 'housing_median_age', 'total_rooms',
       'total_bedrooms', 'population', 'households', 'median_income',
       'ocean_proximity', 'income_cat'],
      dtype='object')

In [41]:
# above we see the list that's in the housing data
# remember for below, lists begin at 0
# ----
# we could do this below, but it's not flexible. instead of hardcoding we could
# assign the variables to the columns
# room_ix, bedrooms_ix, population_ix, households_ix = 3, 4, 5, 6
rooms_ix, bedrooms_ix, population_ix, household_ix = [
    # indexing housing.columns with col
    list(housing.columns).index(col)
    # for col function in this columns
    for col in ('total_rooms', 'total_bedrooms', 'population', 'households')
]

In [47]:
# custom transformer class
class CombinedAttributesAdder(BaseEstimator, TransformerMixin):
    # this is creating a hyperparameter,
    # used to allow us to determine if its helping the ML algorithm or not
    # Hyperparameters are important because they directly control the 
    # behaviour of the training algorithm
    # ---
    # it's used to gate any data you are not sure about
    # ---
    # The process of finding most optimal hyperparameters in machine learning
    # is called hyperparameter optimisation
    def __init__(self, add_bedrooms_per_room = True):
        self.add_bedrooms_per_room = add_bedrooms_per_room
    def fit(self, X, y=None):
        return self # nothing else to do
    def transform(self, X):
        rooms_per_household = X[:, rooms_ix] / X[:, household_ix]
        population_per_household = X[:, population_ix] / X[:, household_ix]
        if self.add_bedrooms_per_room:
            bedrooms_per_room = X[:, bedrooms_ix] / X[:, rooms_ix]
            # numpy.c_ = <numpy.lib.index_tricks.CClass object>
            # Translates slice objects to concatenation along the second axis.
            return np.c_[X, rooms_per_household, population_per_household,
                        bedrooms_per_room]
        else:
            return np.c_[X, rooms_per_household, population_per_household]

# attr_adder equals the class we made above
attr_adder = CombinedAttributesAdder(add_bedrooms_per_room=False)
# adding housing values transformed to attr_adder
housing_extra_attribs = attr_adder.transform(housing.values)

In [56]:
# creating a pipeline
num_pipeline = Pipeline([
    # 'median' chooses how to replace missing values along each column
    ('imputer', SimpleImputer(strategy='median')),
    
    ('addtribs_adder', CombinedAttributesAdder()),
    ('std_scalar', StandardScaler())
])

In [61]:
housing_num = housing.drop('ocean_proximity', axis=1)
imputer = SimpleImputer(strategy="median")
imputer.fit(housing_num)

SimpleImputer(strategy='median')

In [63]:
# all but the last estimator must be transformers
# reason why it's fit_transform()
housing_num_tr = num_pipeline.fit_transform(housing_num)

In [66]:
# using column transformer, which handles categorical and numerical columns
num_attribs = list(housing_num)
cat_attribs = ['ocean_proximity']

full_pipeline = ColumnTransformer([
    ('num', num_pipeline, num_attribs),
    ('cat', OneHotEncoder(), cat_attribs)
])

housing_prepared = full_pipeline.fit_transform(housing)

In [68]:
# --------- LINEAR REGRESSION MODEL -----------
lin_reg = LinearRegression()
# - below is a reminder what housing_labels is
# housing = strat_train_set.drop('median_house_value', axis=1)
# housing_labels = strat_train_set['median_house_value'].copy()
lin_reg.fit(housing_prepared, housing_labels)

LinearRegression()

In [70]:
# with the linear regression model above we can try it out on a full few instances
some_data = housing.iloc[:5]
some_labels = housing_labels.iloc[:5]
some_data_prepared = full_pipeline.transform(some_data)
# here we transformed some_data_prepared from some_data, which is the last 5
# values of housing data. we then used lin_reg.predict, to predict these values
# these predictions are not accurate, apparently by 40%
print('Predictions:', lin_reg.predict(some_data_prepared))

Predictions: [ 83811.08312039 314388.27147935 145641.85902864 183704.64921202
 238729.66725745]


In [71]:
# here we are printing the last 5 actual values
print('Labels:', list(some_labels))

Labels: [72100.0, 279600.0, 82700.0, 112500.0, 238300.0]


In [73]:
# we are going to measure regression model's RMSE
# RMSE = root mean square error
# using scikit-learn's mean_squared_error() function
housing_predictions = lin_reg.predict(housing_prepared)
# the number below creates the typical prediction error
# meaning it's finding the error between housing_labels(actual)
# and the prediction made by lin_reg.predict prediction of housing_prepared
lin_mse = mean_squared_error(housing_labels, housing_predictions)
lin_rmse = np.sqrt(lin_mse)
lin_rmse

68376.51254853733

In [75]:
# we realized the above regression model underfits out data
# we will now try to a more powerful model 'DecisionTreeRegressor'
tree_reg = DecisionTreeRegressor()
tree_reg.fit(housing_prepared, housing_labels)

DecisionTreeRegressor()

In [78]:
# below gives 0.0 as an error result, but we need to be sure it's not overfitted
housing_predictions = tree_reg.predict(housing_prepared)
tree_mse = mean_squared_error(housing_labels, housing_predictions)
tree_rmse = np.sqrt(tree_mse)
tree_rmse

0.0

In [82]:
# creating a cross-validation model to evaluate the tree_reg model above
# creating 10 different folds of the data and training 9 with 1 test set, all
# picked at random
# ----
# scoring="neg_mean_squared_error" will return negative output values
# Assume if MSE is 5 it will return -5. If MSE is 9 it will return -9. 
# This is because the cross_val_score function works on the maximization
# All scorer objects follow the convention that higher return values are 
# better than lower return values
scores = cross_val_score(tree_reg, housing_prepared, housing_labels,
                        scoring="neg_mean_squared_error", cv=10)
tree_rmse_scores = np.sqrt(-scores)# '-' is to compute neg before calculating

In [84]:
# we can look at the results we just did
def display_scores(scores):
    print("Scores: ", scores),
    print("Mean: ", scores.mean()),
    print("Standard Deviation: ", scores.std())

In [85]:
display_scores(tree_rmse_scores)

Scores:  [72183.8769435  69376.696752   68514.59336243 71951.50675594
 68927.5684775  75202.84324676 69779.8559149  72334.10211771
 67641.90080083 70078.15826326]
Mean:  70599.11026348258
Standard Deviation:  2164.8609596177407


In [90]:
# we're going to try this same test with lin_reg
# we can compare the linear and decision tree model, and the decision tree model
# is much more accurate, the linear model it roughly 40%-50% less accurate
lin_scores = cross_val_score(lin_reg, housing_prepared, housing_labels,
                          scoring='neg_mean_squared_error', cv=10)
lin_rsme_scores = np.sqrt(-lin_scores)
display_scores(lin_rsme_scores)

Scores:  [71523.78333874 64044.46774989 67454.97869698 68514.10137273
 66303.62531226 72166.63405138 74464.08841381 68570.11804395
 66063.64175868 69870.86192291]
Mean:  68897.63006613276
Standard Deviation:  3002.746127534865


In [111]:
# using RandomForestRegression
# n_estimators=10 is the number of tree in the forest
forest_reg = RandomForestRegressor()
forest_reg.fit(housing_prepared, housing_labels)

RandomForestRegressor()

In [112]:
housing_predictions = forest_reg.predict(housing_prepared)
forest_mse = mean_squared_error(housing_labels, housing_predictions)
forest_rmse = np.sqrt(forest_mse)
forest_rmse

18664.578293264003

In [113]:
forest_scores = cross_val_score(forest_reg, housing_prepared, housing_labels,
                                scoring="neg_mean_squared_error", cv=10)
forest_rmse_scores = np.sqrt(-forest_scores)
display_scores(forest_rmse_scores)

Scores:  [51771.53009845 48962.68323085 47076.90872453 52179.63676766
 47676.74252173 51634.43678429 52265.56619971 49830.40617325
 48190.41516052 53709.36664067]
Mean:  50329.769230167105
Standard Deviation:  2160.7083540138137


In [116]:
# the code below is how i would save my model
# joblib saves your model in a file,
# joblib.dump = saved into a filename
# joblib.load = the object can be reloaded from the file
# joblib.dump(my_model, 'my_model.pkl')
# my_model_loaded = joblib.load('my_model.pkl')

In [144]:
# we can do a grid search to we find a great combination of hyperparameters
# we can use GridSearchCV for that
param_grid = [
    {'n_estimators': [3, 10, 30], 'max_features': [2, 4, 6, 8]},
    # above ^ we evaluate all 12 combinations of 
    # n_estimators * max_features (3*4=12), (in first dict)
    # this will be explained in ch.7
    {'bootstrap': [False], 'n_estimators': [3, 10], 'max_features': [2, 3, 4]},
    # above ^ 2*3 = 6 in the second dict
    # bootstrap hyperparameter set to false instead of true which is default
]

forest_reg = RandomForestRegressor()

# the grid search will explore 12 + 6 = 18 combinations of hyperparameter values
# we train the model 5 times (cv=5), in all there's 90 rounds of training
# 18 * 5 = 90.
grid_search = GridSearchCV(forest_reg, param_grid, cv=5, 
                           scoring='neg_mean_squared_error',
                           return_train_score=True)

grid_search.fit(housing_prepared, housing_labels)

GridSearchCV(cv=5, estimator=RandomForestRegressor(),
             param_grid=[{'max_features': [2, 4, 6, 8],
                          'n_estimators': [3, 10, 30]},
                         {'bootstrap': [False], 'max_features': [2, 3, 4],
                          'n_estimators': [3, 10]}],
             return_train_score=True, scoring='neg_mean_squared_error')

In [145]:
# to get the best combination of parameters
# ---
# since 8 and 30 are the maximum values that were evaluated, could try searching
# again wtih higher values
grid_search.best_params_

{'max_features': 6, 'n_estimators': 30}

In [146]:
# evaluation scores
# we can that the best score (lowest) comes from 6 and 30 slightly
cvres = grid_search.cv_results_
# the mean calculated on all samples where each sample has the same weight
#----
# The function takes in iterables as arguments and returns an iterator. 
# This iterator generates a series of tuples containing elements 
# from each iterable.
for mean_score, params in zip(cvres['mean_test_score'], cvres['params']):
    print(np.sqrt(-mean_score), params)

64062.66394356172 {'max_features': 2, 'n_estimators': 3}
55447.527291026934 {'max_features': 2, 'n_estimators': 10}
53081.50204728615 {'max_features': 2, 'n_estimators': 30}
60560.17066014013 {'max_features': 4, 'n_estimators': 3}
53545.81515310698 {'max_features': 4, 'n_estimators': 10}
51567.517803232564 {'max_features': 4, 'n_estimators': 30}
59819.36812220254 {'max_features': 6, 'n_estimators': 3}
53381.960013244025 {'max_features': 6, 'n_estimators': 10}
50769.49053093028 {'max_features': 6, 'n_estimators': 30}
59539.61096456183 {'max_features': 8, 'n_estimators': 3}
52917.70958938865 {'max_features': 8, 'n_estimators': 10}
50777.131515707864 {'max_features': 8, 'n_estimators': 30}
62694.306885653066 {'bootstrap': False, 'max_features': 2, 'n_estimators': 3}
54413.251190211624 {'bootstrap': False, 'max_features': 2, 'n_estimators': 10}
60207.92231946349 {'bootstrap': False, 'max_features': 3, 'n_estimators': 3}
52936.536213256826 {'bootstrap': False, 'max_features': 3, 'n_estimato

In [148]:
# feature_importances takes our grid search model which creates combinations
# of hyperparameters from forest_reg which uses RandomForestRegressor().
# ----
# We then applied best_estimator
# best_estimator: Estimator that was chosen by the search, i.e. estimator 
# which gave highest score (or smallest loss if specified) on the left out data.
# seems like best estimator chooses which is the best way to estimate
# ----
# feature_importances_:
# they are computed as the mean and standard deviation of accumulation 
# of the impurity decrease within each tree.
# techniques that calculate a score for all the input features for a 
# given model — the scores simply represent the “importance” of each feature
# A higher score means that the specific feature will have a larger effect 
# on the model that is being used to predict a certain variable
feature_importances = grid_search.best_estimator_.feature_importances_
feature_importances

array([6.74857553e-02, 6.20148913e-02, 4.50994152e-02, 1.84969949e-02,
       1.70256832e-02, 1.78860554e-02, 1.62184057e-02, 2.39562672e-01,
       1.34348224e-01, 5.56606507e-02, 1.04952077e-01, 6.83108884e-02,
       9.66185502e-03, 1.35728130e-01, 1.11058129e-04, 2.53573503e-03,
       4.90150860e-03])

In [149]:
# the output below displays their importance scores next to their attribute
# -----
# 'named_transformers_' Read-only attribute to access any transformer by given 
# name. Keys are transformer names and values are the fitted transformer objects
# ----
# 
extra_attribs = ["rooms_per_hhold", "pop_per_hhold", "bedrooms_per_room"]
#cat_encoder = cat_pipeline.named_steps["cat_encoder"] # old solution
cat_encoder = full_pipeline.named_transformers_["cat"] # ('cat', OneHotEncoder(), cat_attribs)
cat_one_hot_attribs = list(cat_encoder.categories_[0])
attributes = num_attribs + extra_attribs + cat_one_hot_attribs # num_attribs = list(housing_num)
sorted(zip(feature_importances, attributes), reverse=True)

[(0.23956267244790944, 'median_income'),
 (0.13572813022403798, 'INLAND'),
 (0.13434822392204088, 'income_cat'),
 (0.10495207657229565, 'pop_per_hhold'),
 (0.06831088837676674, 'bedrooms_per_room'),
 (0.06748575532150962, 'longitude'),
 (0.06201489125634639, 'latitude'),
 (0.055660650730448395, 'rooms_per_hhold'),
 (0.04509941515552039, 'housing_median_age'),
 (0.018496994937866544, 'total_rooms'),
 (0.017886055350542277, 'population'),
 (0.017025683198693034, 'total_bedrooms'),
 (0.01621840572478285, 'households'),
 (0.009661855020863286, '<1H OCEAN'),
 (0.004901508601591803, 'NEAR OCEAN'),
 (0.0025357350294795823, 'NEAR BAY'),
 (0.00011105812930513959, 'ISLAND')]

In [155]:
# ok ok it took a moment but I figured how what's happening, too bad my memory
# isn't great
# anyways, first, grid search is using best estimator method
# second, we drop median house value in x_test because we don't want it 
# when we try to make predictions, but we use it in y_test to 
# calculate the mean_squared_error
# third, we print the error amount of the model
# note: the model is predicting median_house_value
#---------
# we can now evaluate the final model of the test set
# using transform(), not fit_transform() because we don't want to fit a test set
final_model = grid_search.best_estimator_
# we drop the median_house_value category variable
X_test = strat_test_set.drop("median_house_value", axis=1)
y_test = strat_test_set['median_house_value'].copy()

X_test_prepared = full_pipeline.transform(X_test)
# we are using the grid_search.best_estimator to predict the full_pipeline
# X_test
final_predictions = final_model.predict(X_test_prepared)

final_mse = mean_squared_error(y_test, final_predictions)
final_mse = np.sqrt(final_mse)
final_mse

48995.46533130725

In [159]:
# we can create a 95% confidence interval of this model 
# this creates a bottom and top confidence interval
confidence = .95
squared_errors = (final_predictions - y_test)**2
np.sqrt(stats.t.interval(confidence, len(squared_errors) - 1,
                        loc=squared_errors.mean(),
                        scale=stats.sem(squared_errors)))

array([46965.52243362, 50944.58703925])